In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
from PIL import Image
import sys
import random
from utils.sbi import SBI_Dataset
from utils.scheduler import LinearDecayLR
from sklearn.metrics import confusion_matrix, roc_auc_score
import argparse
from utils.logs import log
from utils.funcs import load_json
from datetime import datetime
from tqdm import tqdm
from model import patchwiseDetectorwithfreq
device = torch.device('cuda:2')
import plotly.express as px


In [2]:
PATH = "../models/pretrained_pcl_effnet.pth"
model = patchwiseDetectorwithfreq()
model.load_state_dict(torch.load(PATH))
model.to(device)

Loaded pretrained weights for efficientnet-b0


patchwiseDetectorwithfreq(
  (net): EfficientNet(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d((0, 1, 0, 1))
    )
    (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
          (static_padding): ZeroPad2d((1, 1, 1, 1))
        )
        (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          32, 8, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          8, 32, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_project_conv): Conv2dStaticS

In [5]:
val_dataset=SBI_Dataset(phase='val',image_size=256)
val_loader=torch.utils.data.DataLoader(val_dataset,
                        batch_size=8,
                        shuffle=False,
                        collate_fn=val_dataset.pretrain_collate_fn,
                        num_workers=4,
                        pin_memory=True,
                        worker_init_fn=val_dataset.worker_init_fn
                        )

SBI(val): 1119


In [6]:
features = None
# gt_labels = None
for i, data in enumerate(tqdm(val_loader)):
    img=data['img'].to(device).float()
    labels=data['label'].to(device).long()
    feature = model(img)
    if features is None:
        features=feature.detach().cpu().numpy()
        gt_labels = labels
    else:
        gt_labels = torch.cat([gt_labels, labels])
        features=np.vstack((features, feature.detach().cpu().numpy()))

gt_labels = gt_labels.cpu().numpy()

100%|██████████| 140/140 [00:50<00:00,  2.79it/s]


In [ ]:
from sklearn.manifold import TSNE

tsne_model = TSNE(n_components=2, perplexity=120)
embeddings = tsne_model.fit_transform(features)
# embeddings.kl_divergence_

In [ ]:
tsne_model.kl_divergence_

2.330202102661133

In [ ]:
fig = px.scatter(x=embeddings[:, 0], y=embeddings[:, 1], color=gt_labels)
fig.update_layout(
    title="t-SNE visualization of Custom Classification dataset",
    xaxis_title="First t-SNE",
    yaxis_title="Second t-SNE",
)
fig.show()